In [1]:
import pandas as pd
import numpy as np

def laad_en_schoon_oee_data(file_path):
    tabbladen = ['2', '11', '24', '25', '29', '31']
    all_sheets = []

    print(f"Bezig met verwerken van: {file_path}")

    try:
        # We gebruiken pd.ExcelFile voor betere performance bij meerdere tabbladen
        xls = pd.ExcelFile(file_path)
        
        for sheet in tabbladen:
            if sheet in xls.sheet_names:
                # Inlezen (we gaan ervan uit dat de headers op rij 1 staan)
                df = pd.read_excel(xls, sheet_name=sheet)
                
                # Voeg de lijn toe als bron-identificatie
                df['Lijn'] = sheet
                all_sheets.append(df)
            else:
                print(f"⚠️ Tabblad {sheet} ontbreekt in het bestand.")

        # Voeg alles samen
        full_df = pd.concat(all_sheets, ignore_index=True)

        # --- DATA CLEANING LOGICA ---

        # 1. Datum conversie (DD-MM-YY)
        full_df['DD-MM-YY'] = pd.to_datetime(full_df['DD-MM-YY'], errors='coerce', dayfirst=True)

        # 2. Numerieke waarden (OEE en Gemiddelde)
        # We vervangen de komma door een punt als het een tekst is, daarna naar float
        for col in ['OEE', 'Gemiddelde']:
            if full_df[col].dtype == 'object':
                full_df[col] = full_df[col].str.replace(',', '.').astype(float)

        # 3. Hoeveelheid opschonen
        # Als de punt een duizendtal-scheider is, moeten we die verwijderen
        if full_df['Hoeveelheid'].dtype == 'object':
             full_df['Hoeveelheid'] = full_df['Hoeveelheid'].str.replace('.', '', regex=False).astype(int)

        print("✅ Data succesvol geladen en geschoond.\n")
        
        # --- INSPECTIE OUTPUT ---
        print("Gedetecteerde kolommen en types:")
        print(full_df.dtypes)
        print("\nEerste 3 rijen van de opgeschoonde data:")
        print(full_df.head(3))
        
        return full_df

    except Exception as e:
        print(f"❌ Er is een fout opgetreden: {e}")
        return None

# Voer het script uit
bestandsnaam = 'Data Lijnen boven OEE .xlsx'
df_oee = laad_en_schoon_oee_data(bestandsnaam)

Bezig met verwerken van: Data Lijnen boven OEE .xlsx
✅ Data succesvol geladen en geschoond.

Gedetecteerde kolommen en types:
DD-MM-YY                 datetime64[ns]
Datum                            object
OEE                             float64
Gemiddelde                      float64
Bandleidster                     object
Hoeveelheid                     float64
Product                          object
Aantal personen                  object
Lijn                             object
Week                            float64
Hoeveelheid productie           float64
dtype: object

Eerste 3 rijen van de opgeschoonde data:
    DD-MM-YY                        Datum    OEE  Gemiddelde Bandleidster  \
0 2025-09-09     dinsdag 9 september 2025  47.13       60.87      Shirley   
1 2025-09-10   woensdag 10 september 2025  23.47       60.87      Shirley   
2 2025-09-11  donderdag 11 september 2025  43.82       60.87      Shirley   

   Hoeveelheid                 Product Aantal personen Lijn  Week  \
